In [1]:
import os
import easyocr
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import time  
import pandas as pd

In [2]:
# Function to display the image
def display_image(img, img_name=None, figsize=(10, 10)):
    """
    Displays an image with optional title and customizable size.
    """
    plt.figure(figsize=figsize)  # Set the figure size
    plt.imshow(img)
    plt.axis('off')  # Turn off axis for better visualization
    if img_name:
        plt.title(img_name)  # Optional: Add a title
    plt.show()

In [14]:
# OCR Processing Function with Confidence Threshold and Bounding Box Coordinates
def process_image(image_path, output_dir, reader, confidence_threshold=0.8):
    """
    Processes a single image using EasyOCR with a confidence threshold.
    Annotates the image with bounding boxes and saves OCR results to a CSV file.
    """
    # Start timing
    start_time = time.time()

    # Load the image
    image = Image.open(image_path)

    # Perform OCR
    results = reader.readtext(image_path)

    # Annotate the image
    annotated_image = image.copy()
    draw = ImageDraw.Draw(annotated_image)
    text_output = []  # Initialize a list to hold OCR result data for CSV

    for (bbox, text, confidence) in results:
        # Filter results based on confidence threshold
        if confidence >= confidence_threshold:
            # Draw bounding box
            top_left = tuple(bbox[0])
            bottom_right = tuple(bbox[2])
            draw.rectangle([top_left, bottom_right], outline="blue", width=3)
            
            # Annotate text
            draw.text((top_left[0], bottom_right[1] + 5), text, fill="red")  # Place text below the bounding box
            
            # Append text, confidence, and bounding box to the output
            text_output.append([text, bbox, confidence])

    # Generate output file names
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    annotated_image_path = os.path.join(output_dir, f"Annotated_{base_name}.png")
    csv_output_path = os.path.join(output_dir, f"OCR_{base_name}.csv")

    # Save annotated image
    annotated_image.save(annotated_image_path)

    # Save text output to CSV
    if text_output:
        df = pd.DataFrame(text_output, columns=["Text", "Bounding Box", "Confidence"])
        df.to_csv(csv_output_path, index=False, encoding='utf-8')

    # End timing and calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Processed: {image_path} in {elapsed_time:.2f} seconds")


In [15]:
# Main script to process all PNG images
def process_all_images(input_dir, output_dir, confidence_threshold=0.8, reader_language="en"):
    """
    Processes all PNG images in the input directory with a confidence threshold.
    Saves annotated images and OCR results to the output directory in CSV format.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Initialize EasyOCR reader
    reader = easyocr.Reader([reader_language])

    # Iterate over all PNG files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.lower().endswith('.png'):
            image_path = os.path.join(input_dir, file_name)
            process_image(image_path, output_dir, reader, confidence_threshold)


In [17]:
image_files = r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images"
# Process all images with a confidence threshold of 0.0 for Turkish
output_dir_tr_0 = r'C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Processed_TR_0_Conf'
process_all_images(image_files, output_dir_tr_0, confidence_threshold=0.0, reader_language="tr")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\adaptive_gaussian.png in 5.01 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\adaptive_mean.png in 5.89 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\binary_threshold_60.png in 5.15 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\binary_threshold_70.png in 5.73 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\binary_threshold_80.png in 6.24 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\binary_threshold_90.png in 6.21 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\easyocr_output.png in 8.37 seconds
Processed: C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Processed images\local_threshold.png in 5.29 seconds
Processed: C:\Users\AZMI\

In [20]:
import os
import pandas as pd
from jiwer import wer

# Path to ground truth words file
ground_truth_path = r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\Compare\words.txt"

# Read ground truth words
with open(ground_truth_path, 'r', encoding='utf-8') as file:
    ground_truth_words = file.read().strip()

# List of CSV file paths (replace with your actual list of paths)
csv_files = os.listdir(r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf")

# Result list for WER calculation
results = []

# Process each CSV file
for csv_file in csv_files:
    csv_file = os.path.join(r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf", csv_file)
    if os.path.isfile(csv_file):  # Check if the file exists
        try:
            # Read the CSV file
            df = pd.read_csv(csv_file)

            # Concatenate all texts in the 'Text' column
            detected_text = " ".join(df['Text'].astype(str).tolist())

            # Calculate WER
            file_wer = wer(ground_truth_words, detected_text)

            # Append results
            results.append({"File Name": os.path.basename(csv_file), "WER": file_wer})
        except Exception as e:
            print(f"Error processing file {csv_file}: {e}")
    else:
        print(f"File not found: {csv_file}")

# Create a DataFrame for results
results_df = pd.DataFrame(results)

# Save results to a new CSV file
output_path = r"C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\WER_Results.csv"
results_df.to_csv(output_path, index=False)

print(f"WER calculation complete. Results saved to {output_path}.")


Error processing file C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf\Annotated_adaptive_gaussian.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error processing file C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf\Annotated_adaptive_mean.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error processing file C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf\Annotated_binary_threshold_60.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error processing file C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf\Annotated_binary_threshold_70.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error processing file C:\Users\AZMI\Desktop\Izzet Ahmet\Kodlar\OCR\Preprocess\EasyOCR\Processed_TR_0_Conf\Annotated_binary_threshold_80.png: 'utf-8' codec can't